In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statistics

## Loading Data

In [2]:
#Load train data
train_data = pd.read_csv('data/train_data.csv')
train_data.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50


## Data Pre-Processing

In [3]:
#Data Types of Different Columns
train_data.dtypes

case_id                                int64
Hospital_code                          int64
Hospital_type_code                    object
City_Code_Hospital                     int64
Hospital_region_code                  object
Available Extra Rooms in Hospital      int64
Department                            object
Ward_Type                             object
Ward_Facility_Code                    object
Bed Grade                            float64
patientid                              int64
City_Code_Patient                    float64
Type of Admission                     object
Severity of Illness                   object
Visitors with Patient                  int64
Age                                   object
Admission_Deposit                    float64
Stay                                  object
dtype: object

In [4]:
train_data.describe()

,case_id,Hospital_code,City_Code_Hospital,Available Extra Rooms in Hospital,Bed Grade,patientid,City_Code_Patient,Visitors with Patient,Admission_Deposit
count,318438.000000,318438.000000,318438.000000,318438.000000,318325.000000,318438.000000,313906.000000,318438.000000,318438.000000
mean,159219.500000,18.318841,4.771717,3.197627,2.625807,65747.579472,7.251859,3.284099,4880.749392
std,91925.276848,8.633755,3.102535,1.168171,0.873146,37979.936440,4.745266,1.764061,1086.776254
min,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1800.000000
25%,79610.250000,11.000000,2.000000,2.000000,2.000000,32847.000000,4.000000,2.000000,4186.000000
50%,159219.500000,19.000000,5.000000,3.000000,3.000000,65724.500000,8.000000,3.000000,4741.000000
75%,238828.750000,26.000000,7.000000,4.000000,3.000000,98470.000000,8.000000,4.000000,5409.000000
max,318438.000000,32.000000,13.000000,24.000000,4.000000,131624.000000,38.000000,32.000000,11008.000000


In [5]:
#Searching for null values
train_data.isna().sum()

case_id                                 0
Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                             113
patientid                               0
City_Code_Patient                    4532
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
Stay                                    0
dtype: int64

In [6]:
#Fill null values with 0's
train_data['Bed Grade'] = train_data['Bed Grade'].fillna(0)
train_data['City_Code_Patient'] = train_data['City_Code_Patient'].fillna(0)

In [7]:
#Confirmation that there are no more null values
train_data.isna().sum()

case_id                              0
Hospital_code                        0
Hospital_type_code                   0
City_Code_Hospital                   0
Hospital_region_code                 0
Available Extra Rooms in Hospital    0
Department                           0
Ward_Type                            0
Ward_Facility_Code                   0
Bed Grade                            0
patientid                            0
City_Code_Patient                    0
Type of Admission                    0
Severity of Illness                  0
Visitors with Patient                0
Age                                  0
Admission_Deposit                    0
Stay                                 0
dtype: int64

In [8]:
#Dropping irrelevant columns
train_data.drop(['case_id', 'patientid'], axis=1, inplace=True)

In [9]:
#Exploring the Stay Column
train_data['Stay'].value_counts()

21-30                 87491
11-20                 78139
31-40                 55159
51-60                 35018
0-10                  23604
41-50                 11743
71-80                 10254
More than 100 Days     6683
81-90                  4838
91-100                 2765
61-70                  2744
Name: Stay, dtype: int64

In [10]:
#Replace the more than 100 days category to something more uniform
train_data['Stay'].replace('More than 100 Days', '100+', inplace=True)

In [11]:
train_data['Stay'].value_counts()

21-30     87491
11-20     78139
31-40     55159
51-60     35018
0-10      23604
41-50     11743
71-80     10254
100+       6683
81-90      4838
91-100     2765
61-70      2744
Name: Stay, dtype: int64

## Feature Engineering

In [12]:
#Divide columns into categorical and numerical
categorical_columns = []
numerical_columns = []

for column in train_data.columns:
    if train_data[column].dtypes == 'object':
        categorical_columns.append(column)
        
for column in train_data.columns:
    if train_data[column].dtypes != 'object':
        numerical_columns.append(column)
        
print(categorical_columns)
print(numerical_columns)

['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness', 'Age', 'Stay']
['Hospital_code', 'City_Code_Hospital', 'Available Extra Rooms in Hospital', 'Bed Grade', 'City_Code_Patient', 'Visitors with Patient', 'Admission_Deposit']


In [13]:
#Some columns in the numerical columns are list are in reality more like categorical variables
categorical_columns.append('Bed Grade')
categorical_columns.append('Hospital_code')
categorical_columns.append('City_Code_Hospital')
categorical_columns.append('City_Code_Patient')

In [14]:
#Remove the above columns from the numerical columns list
numerical_columns.remove('Bed Grade')
numerical_columns.remove('Hospital_code')
numerical_columns.remove('City_Code_Hospital')
numerical_columns.remove("City_Code_Patient")
numerical_columns

['Available Extra Rooms in Hospital',
 'Visitors with Patient',
 'Admission_Deposit']

In [15]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

for column in categorical_columns:
    train_data[column] = label_encoder.fit_transform(train_data[column])

In [16]:
train_data[categorical_columns]

,Hospital_type_code,Hospital_region_code,Department,Ward_Type,Ward_Facility_Code,Type of Admission,Severity of Illness,Age,Stay,Bed Grade,Hospital_code,City_Code_Hospital,City_Code_Patient
0,2,2,3,2,5,0,0,5,0,2,7,2,7
1,2,2,3,3,5,1,0,5,5,2,1,4,7
2,4,0,1,3,4,1,0,5,4,2,9,0,7
3,1,1,3,2,3,1,0,5,5,2,25,1,7
4,1,1,3,3,3,1,0,5,5,2,25,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
318433,0,0,3,1,5,0,2,4,2,4,5,5,22
318434,0,0,1,1,4,2,2,8,4,4,23,0,8
318435,0,0,2,2,5,0,1,7,2,4,6,3,10
318436,1,1,1,1,3,1,1,1,2,3,10,1,8


In [17]:
#Scaling of numerical columns
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train_data[numerical_columns] = scaler.fit_transform(train_data[numerical_columns].values)

In [18]:
train_data.head()

,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,7,2,2,2,-0.169177,3,2,5,2,7,0,0,-0.727923,5,0.027835,0
1,1,2,4,2,-1.025217,3,3,5,2,7,1,0,-0.727923,5,0.987556,5
2,9,4,0,0,-1.025217,1,3,4,2,7,1,0,-0.727923,5,-0.124910,4
3,25,1,1,1,-1.025217,3,2,3,2,7,1,0,-0.727923,5,2.200319,5
4,25,1,1,1,-1.025217,3,3,3,2,7,1,0,-0.727923,5,0.623175,5


In [19]:
#Declaring x and y variables
y_train = train_data['Stay']
X_train = train_data.drop('Stay', axis=1)

## Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_classifier = LogisticRegression(max_iter=5000)
lr_classifier

In [ ]:
lr_classifier.fit(X_train, y_train)

In [ ]:
lr_classifier.score(X_train, y_train)

# Decision Tree

In [20]:
from sklearn.ensemble import RandomForestClassifier

#Fit to the train data
rfc = RandomForestClassifier(n_estimators=200)
rfc = rfc.fit(X_train, y_train)
rfc.score(X_train, y_train)

0.9995163893756398

In [40]:
#Set up grid search parameters
param_grid = {
    'max_depth': [i for i in range(5, 10)],
    'n_estimators': [i for i in range(200, 210) if i % 2 == 0]
}

In [41]:
#Applying grid search
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(rfc, param_grid, verbose=3)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ..................max_depth=5, n_estimators=200; total time=  30.0s
[CV 2/5] END ..................max_depth=5, n_estimators=200; total time=  28.8s
[CV 3/5] END ..................max_depth=5, n_estimators=200; total time=  28.6s
[CV 4/5] END ..................max_depth=5, n_estimators=200; total time=  28.5s
[CV 5/5] END ..................max_depth=5, n_estimators=200; total time=  28.0s
[CV 1/5] END ..................max_depth=5, n_estimators=202; total time=  29.2s
[CV 2/5] END ..................max_depth=5, n_estimators=202; total time=  29.7s
[CV 3/5] END ..................max_depth=5, n_estimators=202; total time=  29.2s
[CV 4/5] END ..................max_depth=5, n_estimators=202; total time=  29.8s


KeyboardInterrupt: 

# Preparing Test Data

In [21]:
#Setting up the test data
test_data = pd.read_csv('data/test_data.csv')
test_data.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit
0,318439,21,c,3,Z,3,gynecology,S,A,2.0,17006,2.0,Emergency,Moderate,2,71-80,3095.0
1,318440,29,a,4,X,2,gynecology,S,F,2.0,17006,2.0,Trauma,Moderate,4,71-80,4018.0
2,318441,26,b,2,Y,3,gynecology,Q,D,4.0,17006,2.0,Emergency,Moderate,3,71-80,4492.0
3,318442,6,a,6,X,3,gynecology,Q,F,2.0,17006,2.0,Trauma,Moderate,3,71-80,4173.0
4,318443,28,b,11,X,2,gynecology,R,F,2.0,17006,2.0,Trauma,Moderate,4,71-80,4161.0


In [22]:
#Column Types
test_data.dtypes

case_id                                int64
Hospital_code                          int64
Hospital_type_code                    object
City_Code_Hospital                     int64
Hospital_region_code                  object
Available Extra Rooms in Hospital      int64
Department                            object
Ward_Type                             object
Ward_Facility_Code                    object
Bed Grade                            float64
patientid                              int64
City_Code_Patient                    float64
Type of Admission                     object
Severity of Illness                   object
Visitors with Patient                  int64
Age                                   object
Admission_Deposit                    float64
dtype: object

In [25]:
#Dropping of irrelevant columns
test_data.drop(['case_id', 'patientid'], axis=1, inplace=True)

In [26]:
#See which columns have null values
test_data.isna().sum()

Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                              35
City_Code_Patient                    2157
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
dtype: int64

In [27]:
#Fill null columns with 0 value
test_data['Bed Grade'] = test_data['Bed Grade'].fillna(0)
test_data['City_Code_Patient'] = test_data['City_Code_Patient'].fillna(0)

In [28]:
#Confirm
test_data.isna().sum()

Hospital_code                        0
Hospital_type_code                   0
City_Code_Hospital                   0
Hospital_region_code                 0
Available Extra Rooms in Hospital    0
Department                           0
Ward_Type                            0
Ward_Facility_Code                   0
Bed Grade                            0
City_Code_Patient                    0
Type of Admission                    0
Severity of Illness                  0
Visitors with Patient                0
Age                                  0
Admission_Deposit                    0
dtype: int64

In [29]:
#Divide columns into categorical and numerical
categorical_columns_test = []
numerical_columns_test = []

for column in test_data.columns:
    if test_data[column].dtypes == 'object':
        categorical_columns_test.append(column)
        
for column in test_data.columns:
    if test_data[column].dtypes != 'object':
        numerical_columns_test.append(column)
        
print(categorical_columns_test)
print(numerical_columns_test)


['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness', 'Age']
['Hospital_code', 'City_Code_Hospital', 'Available Extra Rooms in Hospital', 'Bed Grade', 'City_Code_Patient', 'Visitors with Patient', 'Admission_Deposit']


In [30]:
#Convert some of the numeric ones to categorical
categorical_columns_test.append('Bed Grade')
categorical_columns_test.append('Hospital_code')
categorical_columns_test.append('City_Code_Hospital')
categorical_columns_test.append('City_Code_Patient')

In [31]:
#Remove the above columns from the numeric list
numerical_columns_test.remove('Bed Grade')
numerical_columns_test.remove('Hospital_code')
numerical_columns_test.remove('City_Code_Hospital')
numerical_columns_test.remove("City_Code_Patient")
numerical_columns_test

['Available Extra Rooms in Hospital',
 'Visitors with Patient',
 'Admission_Deposit']

In [32]:
#Label Encode categorical columns in test_data
for column in categorical_columns_test:
    test_data[column] = label_encoder.fit_transform(test_data[column])

In [35]:
test_data[categorical_columns_test]

,Hospital_type_code,Hospital_region_code,Department,Ward_Type,Ward_Facility_Code,Type of Admission,Severity of Illness,Age,Bed Grade,Hospital_code,City_Code_Hospital,City_Code_Patient
0,2,2,2,3,0,0,2,7,2,20,2,2
1,0,0,2,3,5,1,2,7,2,28,3,2
2,1,1,2,1,3,0,2,7,4,25,1,2
3,0,0,2,1,5,1,2,7,2,5,5,2
4,1,0,2,2,5,1,2,7,2,27,9,2
...,...,...,...,...,...,...,...,...,...,...,...,...
137052,1,1,1,1,3,0,1,4,3,10,1,3
137053,4,0,3,2,4,0,2,0,4,24,0,7
137054,2,2,1,2,0,2,1,0,4,29,2,12
137055,0,0,1,2,4,1,1,4,4,4,0,10


In [36]:
#Scale numeric columns in test data
test_data[numerical_columns_test] = scaler.fit_transform(test_data[numerical_columns_test].values)
test_data.head()

,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit
0,20,2,2,2,-0.165503,2,3,0,2,2,0,2,-0.722758,7,-1.642110
1,28,0,3,0,-1.024428,2,3,5,2,2,1,2,0.402568,7,-0.788083
2,25,1,1,1,-0.165503,2,1,3,4,2,0,2,-0.160095,7,-0.349504
3,5,0,5,0,-0.165503,2,1,5,2,2,1,2,-0.160095,7,-0.644666
4,27,1,9,0,-1.024428,2,2,5,2,2,1,2,0.402568,7,-0.655769


In [ ]:
#Declaring x and y variables
X_test = test_data

# K Nearest Neighbor

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
train_scores = []

for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    train_score = knn.score(X_train, y_train)
    train_scores.append(train_score)
    print(f"k: {k}, Score: {train_score:.3f}")
    
plt.plot(range(1, 20, 2), train_scores, marker='o')
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")
plt.show()        

In [ ]:
knn = KNeighborsClassifier(n_neighbors=13 )
knn.fit(X_train, y_train)
print('k=13 Test Acc: %.3f' % knn.score(X_train, y_train))